In [1]:
import pandas as pd
import numpy as np

In [2]:
house_info = pd.read_csv(r'C:\Users\Виталий\PycharmProjects\home_work_1\housing.csv')

In [3]:
dataset = house_info[house_info.ocean_proximity.isin(['<1H OCEAN', 'INLAND'])]
dataset = dataset[['latitude', 'longitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']]
dataset

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
701,37.64,-121.97,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0
830,37.61,-121.99,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0
859,37.57,-121.97,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0
860,37.58,-121.96,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0
861,37.58,-121.98,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0
...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [4]:
dataset.dtypes

latitude              float64
longitude             float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
dtype: object

## Question 1

In [5]:
dataset.isna().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

## Question 2

In [6]:
print("Median value for variable 'population': " + str(dataset.population.median()))

Median value for variable 'population': 1195.0


## Prepare the dataset

In [7]:
n = len(dataset)

n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test
print(n_train, n_val, n_test)

9413 3137 3137


### shuffle

In [8]:
idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)
idx

array([15183,  4469,  9316, ...,  5390,   860,  7270])

### split

In [9]:
train_ds = dataset.iloc[idx[:n_train]]
valid_ds = dataset.iloc[idx[n_train:n_train + n_val]]
test_ds = dataset.iloc[idx[n_train + n_val:]]

In [10]:
train_ds.head()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
19963,36.23,-119.14,22.0,2935.0,523.0,1927.0,530.0,2.5875,70400.0
5929,34.12,-117.79,16.0,2426.0,426.0,1319.0,446.0,4.8125,224500.0
11377,33.68,-117.97,26.0,3653.0,568.0,1930.0,585.0,5.7301,260900.0
6443,34.10,-118.03,32.0,2668.0,609.0,1512.0,541.0,2.9422,233100.0
17546,37.34,-121.87,39.0,2479.0,541.0,1990.0,506.0,2.4306,289100.0


In [11]:
train_ds = train_ds.reset_index(drop=True)
valid_ds = valid_ds.reset_index(drop=True)
test_ds = test_ds.reset_index(drop=True)
train_ds.head()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,36.23,-119.14,22.0,2935.0,523.0,1927.0,530.0,2.5875,70400.0
1,34.12,-117.79,16.0,2426.0,426.0,1319.0,446.0,4.8125,224500.0
2,33.68,-117.97,26.0,3653.0,568.0,1930.0,585.0,5.7301,260900.0
3,34.10,-118.03,32.0,2668.0,609.0,1512.0,541.0,2.9422,233100.0
4,37.34,-121.87,39.0,2479.0,541.0,1990.0,506.0,2.4306,289100.0


In [12]:
y_train = train_ds.median_house_value.values
y_val = valid_ds.median_house_value.values
y_test = test_ds.median_house_value.values

y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
y_test = np.log1p(y_test)

del train_ds['median_house_value']
del valid_ds['median_house_value']
del test_ds['median_house_value']

## Question 3

In [13]:
def linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [14]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

### filling by 0

In [15]:
valid_ds_filling_zero = valid_ds.fillna(0)
train_ds_filling_zero = train_ds.fillna(0)

In [16]:
w_0, w = linear_regression(train_ds_filling_zero, y_train)
y_pred_val = w_0 + valid_ds_filling_zero.dot(w)
np.round(rmse(y_val, y_pred_val),2)

0.34

### filling by mean

In [17]:
mean = train_ds.total_bedrooms.mean()

valid_ds_filling_mean = valid_ds.fillna(mean)
train_ds_filling_mean = train_ds.fillna(mean)

In [18]:
w_0, w = linear_regression(train_ds_filling_mean, y_train)
y_pred_val = w_0 + valid_ds_filling_mean.dot(w)
np.round(rmse(y_val, y_pred_val),2)

0.34

## Question 4

In [19]:
def regularization_linear_regression(X, y, r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X) 
    XTX = XTX + r * np.eye(XTX.shape[0])
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [20]:
r_list = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]
for r in r_list:
    w_0, w = regularization_linear_regression(train_ds_filling_mean, y_train, r)
    y_pred_val = w_0 + valid_ds_filling_mean.dot(w)
    print(r, np.round(rmse(y_val, y_pred_val),2), w_0)

0 0.34 -9.886133216052006
1e-06 0.34 -9.886112219288602
0.0001 0.34 -9.884033961741856
0.001 0.34 -9.865180702547763
0.01 0.34 -9.680528695695905
0.1 0.34 -8.15417116332868
1 0.34 -3.1633181812751747
5 0.35 -0.8483447453559
10 0.35 -0.4418318681601911


## Question 5

In [21]:
seeds_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
n = len(dataset)
rmse_list = []
for seed in seeds_list:
    idx = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(idx)

    train_ds = dataset.iloc[idx[:n_train]]
    valid_ds = dataset.iloc[idx[n_train:n_train + n_val]]
    test_ds = dataset.iloc[idx[n_train + n_val:]]

    train_ds = train_ds.reset_index(drop=True)
    valid_ds = valid_ds.reset_index(drop=True)
    test_ds = test_ds.reset_index(drop=True)
    
    y_train = train_ds.median_house_value.values
    y_val = valid_ds.median_house_value.values
    y_test = test_ds.median_house_value.values

    y_train = np.log1p(y_train)
    y_val = np.log1p(y_val)
    y_test = np.log1p(y_test)

    del train_ds['median_house_value']
    del valid_ds['median_house_value']
    del test_ds['median_house_value']

    valid_ds_filling_zero = valid_ds.fillna(0)
    train_ds_filling_zero = train_ds.fillna(0)

    w_0, w = linear_regression(train_ds_filling_zero, y_train)
    y_pred_val = w_0 + valid_ds_filling_zero.dot(w)
    rmse_value = np.round(rmse(y_val, y_pred_val),3)
    rmse_list.append(rmse_value)

In [22]:
print(rmse_list)
np.round(np.std(rmse_list), 3)

[0.338, 0.338, 0.338, 0.332, 0.339, 0.343, 0.339, 0.347, 0.351, 0.334]


0.005

## Question 6

In [24]:
n = len(dataset)

n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

idx = np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)

train_ds = dataset.iloc[idx[:n_train]]
valid_ds = dataset.iloc[idx[n_train:n_train + n_val]]
test_ds = dataset.iloc[idx[n_train + n_val:]]

train_ds = pd.concat([train_ds, valid_ds])

train_ds = train_ds.reset_index(drop=True)
test_ds = test_ds.reset_index(drop=True)

y_train = train_ds.median_house_value.values
y_test = test_ds.median_house_value.values

y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

del train_ds['median_house_value']
del test_ds['median_house_value']

test_ds_filling_zero = test_ds.fillna(0)
train_ds_filling_zero = train_ds.fillna(0)

w_0, w = regularization_linear_regression(train_ds_filling_zero, y_train, 0.001)
y_pred_val = w_0 + test_ds_filling_zero.dot(w)

print(np.round(rmse(y_test, y_pred_val),2))

0.33
